In [1]:
%%capture

import kagglehub

# Download latest version
path = kagglehub.model_download("qwen-lm/qwen2.5/transformers/1.5b")


In [8]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/1.5b/1


In [2]:
!pip install transformers
!pip install opendatasets

In [3]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/stpeteishii/inaugural-address")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/stpeteishii/inaugural-address


100%|██████████| 7.19k/7.19k [00:00<00:00, 4.24MB/s]

In [5]:
import pandas as pd
import torch

In [6]:
data=pd.read_csv('/content/inaugural-address/Inaugural Address - Trump 2025.csv')
inputt='Summarize the following text in 40 words or fewer. Return the summary: '+ data.iloc[:,0]
display(inputt[0:5])

,text
0,Summarize the following text in 40 words or fe...
1,Summarize the following text in 40 words or fe...
2,Summarize the following text in 40 words or fe...
3,Summarize the following text in 40 words or fe...
4,Summarize the following text in 40 words or fe...


In [7]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs = train_test_split(
    inputt, test_size=0.5, random_state=42
)

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/root/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/1.5b/1")
model = AutoModelForCausalLM.from_pretrained("/root/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/1.5b/1")

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [13]:
def tokenize_data(inputs, tokenizer, max_length=128):
    input_encodings = tokenizer(
        list(inputs), max_length=max_length, padding=True, truncation=True, return_tensors="pt"
    )
    return input_encodings

train_inputs_enc = tokenize_data(train_inputs, tokenizer)
test_inputs_enc = tokenize_data(test_inputs, tokenizer)

In [14]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
        }

train_dataset = CustomDataset(train_inputs_enc)
test_dataset = CustomDataset(test_inputs_enc)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [15]:
model.eval()

for batch in test_loader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    input_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        repetition_penalty=1.2,
        max_length=50,
        temperature=0.7,
    )
    predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    for input_text, pred in zip(input_texts, predictions):
        print("-" * 50)
        print(f"Input Text: {input_text}")
        print()
        print(f"Summary: {pred}")

    break

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Both `max_new_tokens` (=2048) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------
Input Text: Summarize the following text in 40 words or fewer. Return the summary: We will bring prices down, fill our strategic reserves up again right to the top, and export American energy all over the world.  (Applause.)

Summary: Summarize the following text in 40 words or fewer. Return the summary: We will bring prices down, fill our strategic reserves up again right to the top, and export American energy all over the world.  (Applause.) is a great thing for America.
We'll lower costs, replenish stockpiles fully, expand U.S. oil exports globally.

Summary:
Lowering costs & filling stocks; expanding US oil exports worldwide!
--------------------------------------------------
Input Text: Summarize the following text in 40 words or fewer. Return the summary: That is why each day under our administration of American patriots, we will be working to meet every crisis with dignity and power and strength.  We will move with purpose and s